<a href="https://colab.research.google.com/github/ElizabethDu/CryptoClustering/blob/master/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [962 kB]
Ge

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import avg

# Filter for four-bedroom houses
four_bedroom_houses = home_sales_df.filter(home_sales_df["bedrooms"] == 4)

# Calculate the average price
average_price = four_bedroom_houses.select(avg("price")).first()[0]

# Round to two decimal places and display
rounded_avg_price = round(average_price, 2)
print(rounded_avg_price)


299661.01


In [15]:
from pyspark.sql.functions import avg, round

# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bed_houses = home_sales_df.filter((home_sales_df["bedrooms"] == 3) & (home_sales_df["bathrooms"] == 3))

# Group by the year the home was built and calculate the average price
average_price_per_year = three_bed_houses.groupBy("date_built").agg(round(avg("price"), 2).alias("average_price"))

# Show the results
average_price_per_year.show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [20]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# Filter for homes that meet the specified criteria
filtered_homes = home_sales_df.filter(
    (home_sales_df["bedrooms"] == 3) &
    (home_sales_df["bathrooms"] == 3) &
    (home_sales_df["floors"] == 2) &
    (home_sales_df["sqft_living"] >= 2000)
)

# Group by the year the home was built and calculate the average price
average_price_per_year = filtered_homes.groupBy("date_built").agg(
    round(avg("price"), 2).alias("average_price")
)

# Show the results
average_price_per_year.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|     293965.1|
|      2012|    307539.97|
|      2017|    280317.58|
|      2014|    298264.72|
|      2013|    303676.79|
|      2011|    276553.81|
|      2015|    297609.97|
|      2010|    285010.22|
+----------+-------------+



In [40]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# Calculate the average price per view rating, round it to two decimal places,
# and filter for average prices greater than or equal to $350,000
avg_price_per_view = home_sales_df.groupBy("view").agg(
    round(avg("price"), 2).alias("average_price")
).filter("average_price >= 350000").orderBy("view", ascending=False)

# Show the results
avg_price_per_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.1909942626953125 seconds ---


In [41]:
# 7. Cache the the temporary table home_sales.
home_sales_df.createOrReplaceTempView("home_sales")
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [29]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [42]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Calculate the average price per view rating from cached data
avg_price_per_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    GROUP BY view
    HAVING average_price >= 350000
    ORDER BY view DESC
""")

# Show the results
avg_price_per_view_cached.show()

# Calculate and print the runtime
cached_runtime = time.time() - start_time

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.7486791610717773 seconds ---


In [33]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [35]:
# 11. Read the parquet formatted data.
parquet_df=spark.read.parquet("home_sales")

In [45]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('home_sales_parquet')

In [46]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Calculate the average price per view rating from the Parquet DataFrame
avg_price_per_view_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING average_price >= 350000
    ORDER BY view DESC
""")

# Show the results
avg_price_per_view_parquet.show()

# Calculate and print the runtime
parquet_runtime = time.time() - start_time

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.3363292217254639 seconds ---


In [47]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [48]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False